In [106]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import seaborn as sns
from sklearn.neighbors import NearestNeighbors

In [107]:
from scipy.special import softmax
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd
import xgboost as xgb
from scipy.optimize import LinearConstraint, Bounds, minimize
import warnings
warnings.filterwarnings("ignore", message="delta_grad == 0.0. Check if the approximated function is linear.")

class SchoolManifold():

    def __init__(self, embeddings_df: pd.DataFrame, k = 15, ):
        self.embeddings_df = embeddings_df
        self.embeddings = self.embeddings_df.drop(labels = ["unitid"], axis =1)

        self.k = k

    
    def _compute_neighbors(self, k: int):
        knn = NearestNeighbors(n_neighbors = k, algorithm="auto")
        knn.fit(self.embeddings.values)
        dists, indices = knn.kneighbors(self.embeddings.values)
        self.neighbor_dists = dists
        self.neighbor_indices = indices


    def preprocess_for_target(self, target_df: pd.DataFrame, target_col_ind: int = None, target_col_lab = None, total_column_lab = None):
        if (target_col_ind is not None):
            target_col_lab = target_df.columns[target_col_ind]

        if (target_col_lab is None):
            raise ValueError("Either target_col_ind or target_col_lab must not be None")

        self.total_column_lab = total_column_lab
        merged = pd.merge(self.embeddings_df, target_df, how= "inner", on = "unitid")
        
        self.embeddings = merged[self.embeddings.columns]
        self._compute_neighbors(self.k)

        self.target_df = merged[target_df.columns]
        self.features = self.target_df.drop(labels = ["unitid", target_col_lab], axis = 1)
        orig_columns = self.features.columns
        self.target = self.target_df[target_col_lab]
        self.schools = self.target_df["unitid"]

        inds_of_target_in_original = merged.index.values
        neighbor_inds_of_target_in_original = self.neighbor_indices[inds_of_target_in_original]
        neighbor_dists_of_target_col = self.neighbor_dists[inds_of_target_in_original]

        vals_of_target_col = merged[target_col_lab].values
        neighbor_vals_of_target_col =  vals_of_target_col[neighbor_inds_of_target_in_original]
        
        features_of_target_col = merged[orig_columns].values
        neighbor_features_of_target_col = features_of_target_col[neighbor_inds_of_target_in_original]

        mask = neighbor_vals_of_target_col > 1.1 * vals_of_target_col.reshape(-1, 1)
        neighbor_weights_of_target_col = np.where(mask, neighbor_dists_of_target_col, np.PINF)
        neighbor_weights_of_target_col = softmax(-neighbor_weights_of_target_col, axis = 1)

        self.mask = mask
        self.v_star = (neighbor_vals_of_target_col * neighbor_weights_of_target_col).sum(axis = 1)
        self.weights =  neighbor_weights_of_target_col
        self.neighbor_features = neighbor_features_of_target_col
        self.neighbor_target = neighbor_vals_of_target_col
        
        if (total_column_lab is not None):
            self.total_column = self.features[total_column_lab].values.reshape(-1,1)
            self.total_column_index = list(self.features.columns).index(total_column_lab)
            self.neighbor_features[:, :, self.total_column_index] = self.total_column
        
        self.vstars = np.where(np.isnan(self.v_star), self.target.values, self.v_star)
    

    def compute_deltas(self, reg, mode = "projection", eps = 0.1):
        self.reg = reg
        self.adjusted_features = np.sum(self.neighbor_features * self.weights[:, :, None], axis = 1)
        self.adjusted_features = np.where(np.isnan(self.adjusted_features), self.features.values, self.adjusted_features)

        if mode == "projection":
            # maintaining total spending constraint
            if (self.total_column_lab):
                # all_but_total_column = [x for x in range(self.features.shape[-1]) if x != self.total_column_index]
                # non_total_adjusted_features = self.adjusted_features[:, all_but_total_column]
                # non_total_features = self.features.values[:, all_but_total_column]
                # non_total_adjusted_features = non_total_adjusted_features / np.sum(non_total_adjusted_features, axis = 1).reshape(-1, 1)
                # non_total_adjusted_features = non_total_adjusted_features * np.sum(non_total_features, axis = 1).reshape(-1, 1)
                # self.adjusted_features[:, all_but_total_column] =  non_total_adjusted_features
                
                self.adjusted_features_df = pd.DataFrame(self.adjusted_features, columns = self.features.columns)
                self.deltas = self.adjusted_features - self.features.values
            
            if type(reg) == xgb.core.Booster:
                self.vhats = reg.predict(xgb.DMatrix(self.adjusted_features_df))
        
        if mode == "optimize":
            def objective_xgb(x, reg, df_cols):
                x_df = pd.DataFrame(x.reshape(1, -1), columns=list(df_cols))
                x = xgb.DMatrix(x_df)
                v = reg.predict(x)
                return -v
            
            def objective(x, reg, df_cols):
                v = reg.predict(pd.DataFrame(x.reshape(1,-1), columns=list(df_cols)))
                return -v
            d = self.adjusted_features.shape[-1]
            positive_constraint_A = np.eye(d)
            positive_constraint = LinearConstraint(positive_constraint_A, lb = 0, ub = np.inf)
            
            distance_constraint_A = np.eye(d)
            self.vhats = []
            for school, x0 in enumerate(self.features.values):
                if (np.allclose(x0, self.adjusted_features[school])):
                    self.vhats.append(self.target[school])
                    continue
                constraints = []

                vstar = self.vstars[school]

                if (self.total_column_lab is not None):
                    ub = np.zeros(d)
                    lb = 0.3 * x0.flatten()
                    ub[self.total_column_index] = x0[self.total_column_index]
                    other_cols = [x for x in range(d) if x != self.total_column_index]
                    ub[other_cols] = x0[other_cols] + eps * x0[other_cols]
                    
                    bounds = Bounds(lb = lb, ub = ub, keep_feasible=True)
                else:
                    bounds = Bounds(lb = np.zeros(d), ub = x0.flatten() + eps*x0.flatten(), keep_feasible=True)
                
                if type(reg) == xgb.core.Booster:
                    res = minimize(objective_xgb, x0= x0.flatten(), args = (reg, self.features.columns), method="Powell", bounds = bounds)
                else:
                    res = minimize(objective, x0= x0.flatten(), args = (reg, self.features.columns), method="Powell", bounds = bounds)

                self.vhats.append(-res.fun)
                self.adjusted_features[school] = res.x
                print(f"vhat: {-res.fun}, vstar: {vstar}, orig: {self.target[school]}")
                
                
        outcome_results = np.zeros((len(self.vhats), 3))
        outcome_results[:, 0] = self.target[:len(self.vhats)]
        outcome_results[:, 1] = self.vhats[:len(self.vhats)]
        outcome_results[:, 2] = self.vstars[:len(self.vhats)]
        
        outcome_results_df = pd.DataFrame(outcome_results, columns = ["original", "vhat", "vstar"])
        self.adjusted_features_df = pd.DataFrame(self.adjusted_features, columns = self.features.columns)

        self.results = pd.concat((outcome_results_df, self.adjusted_features_df), axis = 1)
        self.deltas = self.adjusted_features - self.features.values

        return self.results

        

### Academic Scores

In [108]:
ic_mca = pd.read_csv("../datasets/Clusters/IC_CLUSTERS_AND_MCA.csv")
ic_mca = ic_mca.loc[:, ic_mca.columns != "ic_cluster"]

academic_scores = pd.read_csv("../datasets/Target Features/score_prediction.csv", index_col=0)
academic_scores.drop(labels=["actcm75_scaled", "satcm75_scaled", "log_sat75_per_dollar_scaled", "log_sat75_per_dollar", "log_act75_per_dollar", "satcm75", "actcm75"], axis = 1, inplace=True)

In [109]:
manifold = SchoolManifold(ic_mca)

In [110]:
manifold.preprocess_for_target(academic_scores, target_col_lab="log_act75_per_dollar_scaled", total_column_lab="Total expenses")

/home/acowlagi/.conda/envs/datathon/lib/python3.9/site-packages/scipy/special/_logsumexp.py:214: RuntimeWarning: invalid value encountered in subtract
  return np.exp(x - logsumexp(x, axis=axis, keepdims=True))


In [111]:
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
import pickle

xgb_reg = xgb.Booster()
xgb_reg.load_model("../models/log_act75_per_dollar_scaled_model.json")
xgb_reg.save_model("../models/log_act75_per_dollar_scaled_model.json")

In [112]:
results = manifold.compute_deltas(xgb_reg, mode = "optimize")

vhat: 1.0439069271087646, vstar: 0.5726446993110135, orig: 0.379413994518644
vhat: 1.3633816242218018, vstar: 0.4125721481358695, orig: 0.0174127467064182
vhat: 1.2649767398834229, vstar: 2.2914104032190394, orig: 0.95829270106815
vhat: 1.2813611030578613, vstar: 0.9249317592710484, orig: 0.7942337692931244
vhat: 0.437824547290802, vstar: -0.09463513189379835, orig: -2.2492695405838523
vhat: 1.1964830160140991, vstar: 0.7868135487640412, orig: 0.1338931574095301
vhat: 1.1668615341186523, vstar: 0.6433147736136492, orig: 0.4977059753331032
vhat: 0.8624697923660278, vstar: 0.8178314145613943, orig: -0.1315432269451632
vhat: 0.5079674124717712, vstar: 0.24688276547128374, orig: 0.0666422797736436
vhat: 1.410454511642456, vstar: 0.5802136645042721, orig: 0.3285035080295177
vhat: 1.1433252096176147, vstar: 0.8278636831518748, orig: -0.4547742083836029
vhat: 1.315432071685791, vstar: 0.7009240333279712, orig: 0.3638892983960661
vhat: 0.591993510723114, vstar: -0.07741608972844571, orig: -0.4

In [ ]:
results.to_csv("../results/student_caliber_results.csv", index = False)

0.9448751051311466

### Plots

In [ ]:
results = pd.read_csv("../results/student_caliber_results.csv")